In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: [Mar 24, 11:59 pm](https://github.com/UBC-CS/cpsc330-2024W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
# !pip uninstall numpy gensim -y
# !pip install numpy==1.24.4 gensim==4.3.1


In [4]:
# !pip install gensim

In [5]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [6]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [7]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [8]:
word_list = ['apple', 'music', 'game', 'machine']
for word in word_list:
    similar_words = glove_wiki_vectors.most_similar(word)
    print('Similar words to', word)
    print(similar_words)

Similar words to apple
[('microsoft', 0.7449405789375305), ('ibm', 0.6821643710136414), ('intel', 0.6778088212013245), ('software', 0.6775422692298889), ('dell', 0.6741442680358887), ('pc', 0.6678153276443481), ('macintosh', 0.66175377368927), ('iphone', 0.6595611572265625), ('ipod', 0.65346759557724), ('hewlett', 0.6516579389572144)]
Similar words to music
[('musical', 0.8128045201301575), ('songs', 0.7978180646896362), ('dance', 0.7896507978439331), ('pop', 0.7862943410873413), ('recording', 0.7650765776634216), ('folk', 0.7602612376213074), ('jazz', 0.7570096850395203), ('concert', 0.7467938661575317), ('artists', 0.732477068901062), ('song', 0.7319164276123047)]
Similar words to game
[('games', 0.8644919991493225), ('play', 0.8316268920898438), ('season', 0.7733194828033447), ('player', 0.7579816579818726), ('players', 0.7287796139717102), ('match', 0.7282912731170654), ('scoring', 0.720136284828186), ('playing', 0.7145521640777588), ('playoffs', 0.7068414092063904), ('team', 0.701

In [9]:
...

Ellipsis

In [10]:
...

Ellipsis

In [11]:
...

Ellipsis

In [12]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [13]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [14]:
coast_shore_similarity = glove_wiki_vectors.similarity('coast', 'shore')
clothes_closet_similarity = glove_wiki_vectors.similarity('clothes', 'closet')
old_new_similarity = glove_wiki_vectors.similarity('old', 'new')
smart_intelligent_similarity = glove_wiki_vectors.similarity('smart', 'intelligent')
dog_cat_similarity = glove_wiki_vectors.similarity('dog', 'cat')
tree_lawyer_similarity = glove_wiki_vectors.similarity('tree', 'lawyer')


In [15]:
coast_shore_similarity 

0.7000272

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [16]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [17]:
for word in test_words:
    in_vocab = word in glove_wiki_vectors
    print(f"{word}: {in_vocab}")


covididiot: False
fomo: False
frenemies: True
anthropause: False
photobomb: False
selfie: False
pxg: False
pacg: False
cct: True
escc: True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [18]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [19]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [20]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283238

In [21]:
glove_wiki_vectors.similarity("white", "success")

0.351824

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

**Example 1**

In [22]:
analogy('man', 'king', 'woman')

man : king :: woman : ?


,Analogy word,Score
0,queen,0.769854
1,monarch,0.684338
2,throne,0.675574
3,daughter,0.659456
4,princess,0.652053
5,prince,0.651703
6,elizabeth,0.646452
7,mother,0.631172
8,emperor,0.610647
9,wife,0.609866


**Example 2**

In [23]:
analogy('china', 'beijing', 'japan')

china : beijing :: japan : ?


,Analogy word,Score
0,tokyo,0.827260
1,seoul,0.749698
2,japanese,0.713126
3,osaka,0.667651
4,korea,0.665625
5,pyongyang,0.644193
6,moscow,0.616781
7,koizumi,0.594192
8,athens,0.592400
9,taipei,0.581650


**Example 3**

In [24]:
glove_wiki_vectors.similarity('black', 'rap')


0.40099174

In [25]:
glove_wiki_vectors.similarity('white', 'rap')

0.22700332

The similarity between 'black' and 'rap' is higher than 'white' and 'rap', this is a bias that was encoded in embedding.

**Example 4**

In [26]:
glove_wiki_vectors.similarity('dog', 'pet')

0.7449565

In [27]:
glove_wiki_vectors.similarity('snake', 'pet')

0.38822758

The similarity between 'dog' and 'pet' is higher than 'snake' and 'pet', which is also stereotypes that dogs are more common pets than snakes.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

1.We can see for example: 
    The similarity between 'black' and 'rap' is higher than 'white' and 'rap', this is a bias that was encoded in embedding.
    The similarity between 'dog' and 'pet' is higher than 'snake' and 'pet', which is also stereotypes that dogs are more common pets than snakes.

    
2.Using biased embeddings in a hiring algorithm could cause it to favor male-associated words over female-associated ones, leading to unfair rejection of qualified female applicants.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [28]:
from sklearn.datasets import fetch_20newsgroups

In [29]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [30]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [31]:
# !pip install spacy

In [32]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [33]:
import re
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "DET", "PART", "ADP", "SPACE", "NUM", "SYM", "X"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """
    

    clean_text = []

    for token in doc:
        if (
            token.is_stop == False  # Check if it's not a stopword
            and token.is_punct == False
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_.lower()  # Take the lemma of the word
            clean_token = re.sub(r'[^a-z\s]', '', lemma)
            clean_text.append(clean_token)
    return " ".join(clean_text)

In [34]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]
df

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read usc wonder help usc provide paragrap...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
...,...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics,look software call shadow know simple raytrace...
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x,archive faq part modified subject patch xr rel...
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns,nice answer question difference fed mandate li...
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x,get gateway dxv try configure xr need correct ...


In [35]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,like subscribe leadership magazine wonder disk...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

Steps:

 - use spacy tokenization and preprocessing on the text
 - remove all urls and emails
 - remove useless tokens
 - excluding stopwords and punctuation
 - lemmatization

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

In [36]:
# docs  = df['text']
# docs

In [37]:
# vectorizer = CountVectorizer(stop_words="english")
# X = vectorizer.fit_transform(docs)
# X


In [38]:
# from sklearn.decomposition import LatentDirichletAllocation
# from sklearn.feature_extraction.text import CountVectorizer
# import numpy as np

# docs= df['text_pp']

# vectorizer = CountVectorizer(stop_words="english")
# X = vectorizer.fit_transform(docs)

# # Try multiple values of n_components
# topic_range = [1,2,3,4, 6, 8, 10]
# perplexities = []

# for n in topic_range:
#     lda = LatentDirichletAllocation(n_components=n, random_state=42)
#     lda.fit(X)
#     perplexity_val = lda.perplexity(X)
#     perplexities.append(perplexity_val)
#     print(f"n_components = {n}, Perplexity = {perplexity_val:.2f}")

# # Plot perplexity
# plt.plot(topic_range, perplexities, marker='o')
# plt.title("LDA Perplexity vs. Number of Topics")
# plt.xlabel("Number of Topics (n_components)")
# plt.ylabel("Perplexity")
# plt.show()


In [39]:
from sklearn.decomposition import LatentDirichletAllocation
vec = CountVectorizer(stop_words="english")
toy_X = vec.fit_transform(df["text"])
import mglearn
for i in range(1,11):
    lda = LatentDirichletAllocation(
        n_components=i, learning_method="batch", max_iter=10, random_state=0
    )
    lda.fit(toy_X) 
    document_topics = lda.transform(toy_X)
    
    sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
    feature_names = np.array(vec.get_feature_names_out())
    
    print('This is results for ' +str(i) +'topic')
    mglearn.tools.print_topics(
        topics=range(i),
        feature_names=feature_names,
        sorting=sorting,
        topics_per_chunk=5,
        n_words=10,
    )

This is results for 1topic
topic 0       
--------      
people        
don           
like          
just          
god           
know          
think         
time          
does          
file          


This is results for 2topic
topic 0       topic 1       
--------      --------      
file          people        
edu           god           
use           don           
team          just          
program       think         
10            know          
game          like          
year          said          
window        say           
like          time          


This is results for 3topic
topic 0       topic 1       topic 2       
--------      --------      --------      
team          people        file          
game          god           use           
year          don           edu           
10            think         program       
play          just          window        
games         know          image         
season        like          available     


We choose n_components = 8 because it makes sense the most out of the tested model

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

After adjusting the proprocessing function, we have reasonable topics division.

 - topic 0: 'graphics pictures' since words contain 'jpeg', 'image', 'display'...
 - topic 1: 'war israel ' since words contain 'israel', 'israeli', 'attack'...
 - topic 2: 'politics guns' since words contain 'gun', 'guns', ...
 - topic 3: 'god' since words contain 'god', 'christ', 'jesus'...
 - topic 4: 'sport' since words contain 'game', 'player', 'team'...
 - topic 5: 'christian' since words contain 'god', 'believe'...
 - topic 6: 'coding' since words contain 'programe', 'file', 'printf'...
 - topic 7: 'server storage' since words contain 'file', 'server' ,'window'


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [40]:
lda = LatentDirichletAllocation(
        n_components=8, learning_method="batch", max_iter=10, random_state=0
    )
lda.fit(toy_X) 
document_topics = lda.transform(toy_X)
document_topics

array([[1.78579632e-03, 1.78749150e-03, 6.96697905e-01, ...,
        1.51642883e-01, 1.78652136e-03, 1.42725416e-01],
       [1.62431704e-03, 1.62488649e-03, 1.62532223e-03, ...,
        7.82018734e-02, 1.62456645e-03, 6.24174819e-01],
       [1.86689412e-03, 1.86639996e-03, 1.86779266e-03, ...,
        6.06126390e-01, 1.86622142e-03, 2.68014512e-01],
       ...,
       [5.99626237e-02, 2.27376740e-03, 6.34998857e-01, ...,
        2.93669570e-01, 2.27328415e-03, 2.27419537e-03],
       [2.72183061e-03, 2.71844995e-03, 5.98389430e-02, ...,
        2.72029262e-03, 9.82793651e-02, 6.77033612e-01],
       [3.64526147e-04, 5.38446079e-01, 2.04602387e-01, ...,
        1.16695743e-01, 3.64689483e-04, 9.27225379e-02]])

In [41]:
topic_names = ['Topic 0', 'Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5', 'Topic 6', 'Topic 7']
document_topics_df = pd.DataFrame(document_topics[:5], columns=topic_names)
print(document_topics_df)

    Topic 0   Topic 1   Topic 2   Topic 3   Topic 4   Topic 5   Topic 6  \
0  0.001786  0.001787  0.696698  0.001787  0.001787  0.151643  0.001787   
1  0.001624  0.001625  0.001625  0.001624  0.289500  0.078202  0.001625   
2  0.001867  0.001866  0.001868  0.116525  0.001867  0.606126  0.001866   
3  0.003208  0.003207  0.003210  0.003209  0.003208  0.003217  0.003205   
4  0.002718  0.680473  0.002721  0.303206  0.002719  0.002723  0.002719   

    Topic 7  
0  0.142725  
1  0.624175  
2  0.268015  
3  0.977536  
4  0.002721  


Based on the topic distribution probabilities and topic keywords, here is the interpretation of which topic each document most likely belongs to:

1. Document 0  
   - Most likely topic: Topic 2 (`0.696698`)  
   - Topic meaning: Politics and guns (keywords: `'gun'`, `'guns'`)  
   - Comment: The document is likely about political or gun-related issues.

2. Document 1  
   - Most likely topic: Topic 1 (`0.680473`)  
   - Topic meaning: War and Israel (keywords: `'israel'`, `'attack'`)  
   - Comment: This document is likely discussing war or geopolitical topics involving Israel.

3. Document 2  
   - Most likely topic: Topic 5 (`0.606126`)  
   - Topic meaning: Christianity (keywords: `'god'`, `'believe'`)  
   - Comment: Likely focuses on Christian beliefs or religion.

4. Document 3  
   - Most likely topic: Topic 7 (`0.977536`)  
   - Topic meaning: Server and storage (keywords: `'file'`, `'server'`, `'window'`)  
   - Comment: Clearly related to computer systems or file storage.

5. Document 4  
   - Most likely topic: Topic 1 (`0.680473`)  
   - Topic meaning: War and Israel  
   - Comment: Similar to Document 1, this likely discusses war-related themes.


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6



1. Content-based filtering recommends items to users based on the similarity between the content of items and the user’s preferences. For example, if a user liked a sci-fi movie, the system will suggest other sci-fi movies with similar features (e.g., genre, keywords, actors).

2. Two negative consequences of recommender systems:
   - Filter bubbles: Users may only be exposed to content that reinforces their existing preferences, limiting diversity of information.
   - Cold-start problem: New users or items have little to no data, making it difficult to generate accurate recommendations.

3. Transfer learning in NLP involves taking a model trained on a large general dataset (like BERT trained on Wikipedia) and fine-tuning it on a smaller, task-specific dataset (like sentiment analysis). This allows models to perform well with less task-specific data and training time.


<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)